# **Uber Visualizations & Dash Webapp**

**Data Collection & Preparation**

In [3]:
import pandas as pd
import numpy as np
from plotly import graph_objs as go
from plotly.graph_objs import *
from datetime import datetime as dt
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [4]:
df1 = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/uber-rides-data1.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/uber-rides-data2.csv")
df3 = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/uber-rides-data3.csv")
df = pd.concat([df1, df2, df3], axis=0)

In [5]:
df.head()

,Date/Time,Lat,Lon
0,2014-04-01 00:11:00,40.7690,-73.9549
1,2014-04-01 00:17:00,40.7267,-74.0345
2,2014-04-01 00:21:00,40.7316,-73.9873
3,2014-04-01 00:28:00,40.7588,-73.9776
4,2014-04-01 00:33:00,40.7594,-73.9722


In [6]:
df["Date/Time"] = pd.to_datetime(df["Date/Time"], format="%Y-%m-%d %H:%M")
df.index = df["Date/Time"]
df.drop("Date/Time", 1, inplace=True)

In [7]:
pick_ups = []

for month in df.groupby(df.index.month):
    daily_pick_ups = []
    for day in month[1].groupby(month[1].index.day):
        daily_pick_ups.append(day[1])
    pick_ups.append(daily_pick_ups)

pick_ups = np.array(pick_ups, dtype=object)

**Visualizations**

In [8]:
def get_coords(month, day):
    list_coords = pick_ups[month][day]
    return list_coords

In [10]:
list_coords = get_coords(4, 30)

In [11]:
zoom = 12.0
initial_lat = 40.7272
initial_lon = -73.991251
bearing = 0

mapbox_access_token = 'pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNrOWJqb2F4djBnMjEzbG50amg0dnJieG4ifQ.Zme1-Uzoi75IaFbieBDl3A'

mapbox = go.Figure(data=[
                Scattermapbox(
                    lat=list_coords["Lat"],
                    lon=list_coords["Lon"],
                    mode="markers",
                    text=list_coords.index.hour,
                    hoverinfo="lat+lon+text",
                    marker=dict(
                        showscale=True,
                        color=np.append(np.insert(list_coords.index.hour, 0, 0), 23),
                        opacity=0.5,
                        size=5,
                        colorscale=[
                                [0, "#F4EC15"],
                                [0.04167, "#DAF017"],
                                [0.0833, "#BBEC19"],
                                [0.125, "#9DE81B"],
                                [0.1667, "#80E41D"],
                                [0.2083, "#66E01F"],
                                [0.25, "#4CDC20"],
                                [0.292, "#34D822"],
                                [0.333, "#24D249"],
                                [0.375, "#25D042"],
                                [0.4167, "#26CC58"],
                                [0.4583, "#28C86D"],
                                [0.50, "#29C481"],
                                [0.54167, "#2AC093"],
                                [0.5833, "#2BBCA4"],
                                [1.0, "#613099"],
                            ],
                        colorbar=dict(
                                title="Time of<br>Day",
                                x=0.93,
                                xpad=0,
                                nticks=24,
                                tickfont=dict(color="#d8d8d8"),
                                titlefont=dict(color="#d8d8d8"),
                                thicknessmode="pixels",
                            ),
                        ),
                    ),
                ],
          layout=Layout(
                    autosize=True,
                    margin=go.layout.Margin(l=0, r=35, t=0, b=0),
                    showlegend=False,
                    mapbox=dict(
                        accesstoken=mapbox_access_token,
                        center=dict(lat=initial_lat, lon=initial_lon),  # 40.7272  # -73.991251
                        style="dark",
                        bearing=bearing,
                        zoom=zoom,
                    ),
                  updatemenus=[
                        dict(
                            buttons=(
                                [
                                    dict(
                                        args=[
                                            {
                                                "mapbox.zoom": 12,
                                                "mapbox.center.lon": "-73.991251",
                                                "mapbox.center.lat": "40.7272",
                                                "mapbox.bearing": 0,
                                                "mapbox.style": "dark",
                                            }
                                        ],
                                        label="Reset Zoom",
                                        method="relayout",
                                    )
                                ]
                            ),
                            direction="left",
                            pad={"r": 0, "t": 0, "b": 0, "l": 0},
                            showactive=False,
                            type="buttons",
                            x=0.45,
                            y=0.02,
                            xanchor="left",
                            yanchor="bottom",
                            bgcolor="#323130",
                            borderwidth=1,
                            bordercolor="#6d6d6d",
                            font=dict(color="#FFFFFF"),
                        )
                    ],
              )
         )

mapbox.show()

In [12]:
def get_selection(month, day):
    x_val = []
    y_val = []

    color_val = [
        "#F4EC15",
        "#DAF017",
        "#BBEC19",
        "#9DE81B",
        "#80E41D",
        "#66E01F",
        "#4CDC20",
        "#34D822",
        "#24D249",
        "#25D042",
        "#26CC58",
        "#28C86D",
        "#29C481",
        "#2AC093",
        "#2BBCA4",
        "#2BB5B8",
        "#2C99B4",
        "#2D7EB0",
        "#2D65AC",
        "#2E4EA4",
        "#2E38A4",
        "#3B2FA0",
        "#4E2F9C",
        "#603099",
    ]
    
    # Get the number of rides at a particular time
    for i in range(24):
        x_val.append(i)
        y_val.append(len(pick_ups[month][day][pick_ups[month][day].index.hour == i]))
    
    return [np.array(x_val), np.array(y_val), np.array(color_val)]

In [13]:
date_picked = "2014-04-30"
date_picked = dt.strptime(date_picked, "%Y-%m-%d")

month_picked = date_picked.month - 4
day_picked = date_picked.day - 1

[x_val, y_val, color_val] = get_selection(month_picked, day_picked)

hist = go.Figure(data=[
                go.Bar(x=x_val, y=y_val, marker=dict(color=color_val), hoverinfo="x"),
                go.Scatter(
                    opacity=0,
                    x=x_val,
                    y=y_val / 2,
                    hoverinfo="none",
                    mode="markers",
                    marker=dict(color="rgb(66, 134, 244, 0)", symbol="square", size=40),
                    visible=True,
                    ),
                ],
          layout = go.Layout(
                    bargap=0.01,
                    bargroupgap=0,
                    barmode="group",
                    margin=go.layout.Margin(l=10, r=0, t=0, b=50),
                    showlegend=False,
                    plot_bgcolor="#323130",
                    paper_bgcolor="#323130",
                    dragmode="select",
                    font=dict(color="white"),
                    xaxis=dict(
                        range=[-0.5, 23.5],
                        showgrid=False,
                        nticks=25,
                        fixedrange=True,
                        ticksuffix=":00",
                    ),
                    yaxis=dict(
                        range=[0, max(y_val) + max(y_val) / 4],
                        showticklabels=False,
                        showgrid=False,
                        fixedrange=True,
                        rangemode="nonnegative",
                        zeroline=False,
                    ),
                    annotations=[
                        dict(
                            x=xi,
                            y=yi,
                            text=str(yi),
                            xanchor="center",
                            yanchor="bottom",
                            showarrow=False,
                            font=dict(color="white"),
                        )
                        for xi, yi in zip(x_val, y_val)
                    ],
                )
        )

hist.show()

**Dash Webapp**

In [15]:
from dash.dependencies import Input, Output

mapbox_access_token = 'pk.eyJ1Ijoibmlha2hhIiwiYSI6ImNraXJiZWl4ZDFxZm8ycXAzNTlvajNrdngifQ.3hKMJ5KpQxH8tTuHzxBeBg'

app = dash.Dash(__name__)

app.layout = html.Div(
    children=[
        html.Div(
            className="row",
            children=[
                # Column for user controls
                html.Div(
                    className="four columns div-user-controls",
                    children=[
                        html.H2("DASH - UBER DATA APP"),
                        html.P(
                            """Select different days using the date picker to update
                            the information displayed by both the map and the histogram"""
                        ),
                        html.Div(
                            className="div-for-dropdown",
                            children=[
                                dcc.DatePickerSingle(
                                    id="date-picker",
                                    min_date_allowed=dt(2014, 4, 1),
                                    max_date_allowed=dt(2014, 9, 30),
                                    initial_visible_month=dt(2014, 4, 1),
                                    date=dt(2014, 4, 1).date(),
                                    display_format="MMMM D, YYYY",
                                    style={"border": "0px solid black"},
                                )
                            ],
                        ),
                    ]
                ),
                # Column for app graphs and plots
                html.Div(
                    className="eight columns div-for-charts bg-grey",
                    children=[
                        dcc.Graph(id="map-graph"),
                        dcc.Graph(id="histogram"),
                    ],
                ),
            ]
        )
    ]
)


@app.callback(
    Output("histogram", "figure"),
    Input("date-picker", "date"),
)
def update_histogram(date_picked):
    
    date_picked = dt.strptime(date_picked, "%Y-%m-%d")
    month_picked = date_picked.month - 4
    day_picked = date_picked.day - 1

    [x_val, y_val, color_val] = get_selection(month_picked, day_picked)

    layout = go.Layout(
        bargap=0.01,
        bargroupgap=0,
        barmode="group",
        margin=go.layout.Margin(l=10, r=0, t=0, b=50),
        showlegend=False,
        plot_bgcolor="#323130",
        paper_bgcolor="#323130",
        dragmode="select",
        font=dict(color="white"),
        xaxis=dict(
            range=[-0.5, 23.5],
            showgrid=False,
            nticks=25,
            fixedrange=True,
            ticksuffix=":00",
        ),
        yaxis=dict(
            range=[0, max(y_val) + max(y_val) / 4],
            showticklabels=False,
            showgrid=False,
            fixedrange=True,
            rangemode="nonnegative",
            zeroline=False,
        ),
        annotations=[
            dict(
                x=xi,
                y=yi,
                text=str(yi),
                xanchor="center",
                yanchor="bottom",
                showarrow=False,
                font=dict(color="white"),
            )
            for xi, yi in zip(x_val, y_val)
        ],
    )

    return go.Figure(
        data=[
            go.Bar(x=x_val, y=y_val, marker=dict(color=color_val), hoverinfo="x"),
            go.Scatter(
                opacity=0,
                x=x_val,
                y=y_val / 2,
                hoverinfo="none",
                mode="markers",
                marker=dict(color="rgb(66, 134, 244, 0)", symbol="square", size=40),
                visible=True,
            ),
        ],
        layout=layout,
    )

@app.callback(
    Output("map-graph", "figure"),
    Input("date-picker", "date"),
)
def update_graph(date_picked):
    zoom = 12.0
    initial_lat = 40.7272
    initial_lon= -73.991251
    bearing = 0

    date_picked = dt.strptime(date_picked, "%Y-%m-%d")
    month_picked = date_picked.month - 4
    day_picked = date_picked.day - 1
    
    list_coords = get_coords(month_picked, day_picked)

    return go.Figure(
        data=[
            Scattermapbox(
                lat=list_coords["Lat"],
                lon=list_coords["Lon"],
                mode="markers",
                hoverinfo="lat+lon+text",
                text=list_coords.index.hour,
                marker=dict(
                    showscale=True,
                    color=np.append(np.insert(list_coords.index.hour, 0, 0), 23),
                    opacity=0.5,
                    size=5,
                    colorscale=[
                        [0, "#F4EC15"],
                        [0.04167, "#DAF017"],
                        [0.0833, "#BBEC19"],
                        [0.125, "#9DE81B"],
                        [0.1667, "#80E41D"],
                        [0.2083, "#66E01F"],
                        [0.25, "#4CDC20"],
                        [0.292, "#34D822"],
                        [0.333, "#24D249"],
                        [0.375, "#25D042"],
                        [0.4167, "#26CC58"],
                        [0.4583, "#28C86D"],
                        [0.50, "#29C481"],
                        [0.54167, "#2AC093"],
                        [0.5833, "#2BBCA4"],
                        [1.0, "#613099"],
                    ],
                    colorbar=dict(
                        title="Time of<br>Day",
                        x=0.93,
                        xpad=0,
                        nticks=24,
                        tickfont=dict(color="#d8d8d8"),
                        titlefont=dict(color="#d8d8d8"),
                        thicknessmode="pixels",
                    ),
                ),
            ),
        ],
        layout=Layout(
            autosize=True,
            margin=go.layout.Margin(l=0, r=35, t=0, b=0),
            showlegend=False,
            mapbox=dict(
                accesstoken=mapbox_access_token,
                center=dict(lat=initial_lat, lon=initial_lon),  # 40.7272  # -73.991251
                style="dark",
                bearing=bearing,
                zoom=zoom,
            ),
            updatemenus=[
                dict(
                    buttons=(
                        [
                            dict(
                                args=[
                                    {
                                        "mapbox.zoom": 12,
                                        "mapbox.center.lon": "-73.991251",
                                        "mapbox.center.lat": "40.7272",
                                        "mapbox.bearing": 0,
                                        "mapbox.style": "dark",
                                    }
                                ],
                                label="Reset Zoom",
                                method="relayout",
                            )
                        ]
                    ),
                    direction="left",
                    pad={"r": 0, "t": 0, "b": 0, "l": 0},
                    showactive=False,
                    type="buttons",
                    x=0.45,
                    y=0.02,
                    xanchor="left",
                    yanchor="bottom",
                    bgcolor="#323130",
                    borderwidth=1,
                    bordercolor="#6d6d6d",
                    font=dict(color="#FFFFFF"),
                )
            ],
        ),
    )


app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
